<a href="https://colab.research.google.com/github/Gabrielcs96/Analise-e-desenvolvimento-de-uma-ferramenta-de-prevencao-a-fraudes/blob/main/Pyspark02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark
# Install the pyspark package.
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=4dcf61d224427d7494e2c58849792df540f70ca2e5e01945dea3b3012a8b43a9
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
from pyspark.sql.window import Window # Window function

Iniciar sessão com o PySpark

In [4]:
spark = (SparkSession.builder
         .master('local')
         .appName('PySpark_02')
         .getOrCreate()
      )

In [5]:
from google.colab import files
uploaded = files.upload()


Saving wc2018-players.csv to wc2018-players.csv


In [6]:
df = spark.read.csv('wc2018-players.csv', header=True, inferSchema=True)

In [7]:
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



In [8]:
df = df.withColumnRenamed('Team', 'Selecao').withColumnRenamed('Position', 'Posicao').withColumnRenamed('Shirt Name', 'Camisa').withColumnRenamed('Age', 'Idade')\
.withColumnRenamed('Height', 'Altura').withColumnRenamed('Weight', 'Peso').withColumnRenamed('FIFA Popular Name', 'Nome_FIFA').withColumnRenamed('#', 'Numero')\
.withColumnRenamed('Birth Date', 'Nascimento').withColumnRenamed('Club', 'Time').withColumnRenamed('Pos.','Posicao')

In [9]:
dia = udf(lambda data: data.split('.')[0])
mes = udf(lambda data: data.split('.')[1])
ano = udf(lambda data: data.split('.')[2])

In [10]:

df = df.withColumn('Dia', dia('Nascimento')).withColumn('Mes', mes('Nascimento')).withColumn('Ano', ano('Nascimento'))
df = df.withColumn('Data_Nascimento', concat_ws('-', 'Ano', 'Mes', 'Dia').cast(DateType())) # Cast to DateType

In [11]:
df.show(5)

+---------+------+-------+------------------+----------+----------+--------------------+------+----+---+---+----+---------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|    Camisa|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|
+---------+------+-------+------------------+----------+----------+--------------------+------+----+---+---+----+---------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|     1990-07-13|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|  

A coluna " nascimento " agora é inútil. Vamos remover ela !

In [12]:
df = df.drop('Nascimento')

In [13]:
#Backup do DF
df2 = df

Vamos começar a  Usar esse Window.particionBy é algo parecido com o Group By do SQL ... O legal aqui é que você consegue usar o " partition " como parte do código para gerar um row number  e enumerar os caras, nesse caso aqui pela altura para cada seleção.




In [14]:
parametro = Window.partitionBy('Selecao').orderBy(col('Altura').desc())
df.withColumn('numero_lina', row_number().over(parametro)).show(50)

+---------+------+-------+------------------+----------+--------------------+------+----+---+---+----+---------------+-----------+
|  Selecao|Numero|Posicao|         Nome_FIFA|    Camisa|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|numero_lina|
+---------+------+-------+------------------+----------+--------------------+------+----+---+---+----+---------------+-----------+
|Argentina|     6|     DF|    FAZIO Federico|     FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|          1|
|Argentina|     1|     GK|     GUZMAN Nahuel|    GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|          2|
|Argentina|    16|     DF|       ROJO Marcos|      ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20|          3|
|Argentina|    12|     GK|     ARMANI Franco|    ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16|          4|
|Argentina|    23|     GK|CABALLERO Wilfredo| CABALLERO|    Chelsea FC (ENG)|   186

In [15]:
parametro2= Window.partitionBy('Selecao').orderBy(col('Numero').asc())
df.withColumn('numero_linha2', row_number().over(parametro2)).show(50)

+---------+------+-------+------------------+------------+--------------------+------+----+---+---+----+---------------+-------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|      Camisa|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|numero_linha2|
+---------+------+-------+------------------+------------+--------------------+------+----+---+---+----+---------------+-------------+
|Argentina|     1|     GK|     GUZMAN Nahuel|      GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|            1|
|Argentina|     2|     DF|   MERCADO Gabriel|     MERCADO|    Sevilla FC (ESP)|   181|  81| 18| 03|1987|     1987-03-18|            2|
|Argentina|     3|     DF|TAGLIAFICO Nicolas|  TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|            3|
|Argentina|     4|     DF|  ANSALDI Cristian|     ANSALDI|     Torino FC (ITA)|   181|  73| 20| 09|1986|     1986-09-20|            4|
|Argentina|     5|     MF|      BIGLIA Lucas|      BIGL

Vamos usar o Rank

In [16]:
rank1 = Window.partitionBy('Selecao').orderBy(desc('Altura'))
df.withColumn('rank',rank().over(rank1)).show(50)

+---------+------+-------+------------------+----------+--------------------+------+----+---+---+----+---------------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|    Camisa|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|rank|
+---------+------+-------+------------------+----------+--------------------+------+----+---+---+----+---------------+----+
|Argentina|     6|     DF|    FAZIO Federico|     FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|   1|
|Argentina|     1|     GK|     GUZMAN Nahuel|    GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|   2|
|Argentina|    16|     DF|       ROJO Marcos|      ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20|   3|
|Argentina|    12|     GK|     ARMANI Franco|    ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16|   3|
|Argentina|    23|     GK|CABALLERO Wilfredo| CABALLERO|    Chelsea FC (ENG)|   186|  80| 28| 09|1981|     1981-09-28|   5|
|Argenti

In [17]:
rank2 = Window.partitionBy('Selecao').orderBy(desc('Ano'))
df.withColumn('rank',dense_rank().over(rank2)).show(50)

+---------+------+-------+------------------+----------+--------------------+------+----+---+---+----+---------------+----+
|  Selecao|Numero|Posicao|         Nome_FIFA|    Camisa|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|rank|
+---------+------+-------+------------------+----------+--------------------+------+----+---+---+----+---------------+----+
|Argentina|    22|     MF|    PAVON Cristian|     PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     1996-01-21|   1|
|Argentina|    20|     MF|  LO CELSO Giovani|  LO CELSO|Paris Saint-Germa...|   177|  75| 09| 04|1996|     1996-04-09|   1|
|Argentina|    15|     MF|    LANZINI Manuel|   LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     1993-02-15|   2|
|Argentina|    21|     FW|      DYBALA Paulo|    DYBALA|   Juventus FC (ITA)|   177|  73| 15| 11|1993|     1993-11-15|   2|
|Argentina|     3|     DF|TAGLIAFICO Nicolas|TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|   3|
|Argenti

Window Function 4  Porcentagem e Rangking - percentent_rangk

In [18]:
porcentagem = Window.partitionBy('Selecao').orderBy(desc('Altura'))
df.withColumn('porcentagem', percent_rank().over(porcentagem)).show(50)


+---------+------+-------+------------------+----------+--------------------+------+----+---+---+----+---------------+--------------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|    Camisa|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|         porcentagem|
+---------+------+-------+------------------+----------+--------------------+------+----+---+---+----+---------------+--------------------+
|Argentina|     6|     DF|    FAZIO Federico|     FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|                 0.0|
|Argentina|     1|     GK|     GUZMAN Nahuel|    GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|0.045454545454545456|
|Argentina|    16|     DF|       ROJO Marcos|      ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20| 0.09090909090909091|
|Argentina|    12|     GK|     ARMANI Franco|    ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16| 0.09090909090909091|
|Argentina|    23|  

Eindow Function  5- Divisão em "N" partes - ntile()

---
Essa função é boa para agrupar em grupos, no caso abaixo, vamos dividir os grupos por alturas. Como aqi, nós pegamos 5 partes dentro do ntile , a gente vai colher elees quebrados em 5 partes



In [25]:

parte = Window.partitionBy('Selecao').orderBy(desc('Altura'))
df.withColumn('Par',ntile(5).over(parte)).show(50)

+---------+------+-------+------------------+----------+--------------------+------+----+---+---+----+---------------+---+
|  Selecao|Numero|Posicao|         Nome_FIFA|    Camisa|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|Par|
+---------+------+-------+------------------+----------+--------------------+------+----+---+---+----+---------------+---+
|Argentina|     6|     DF|    FAZIO Federico|     FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|  1|
|Argentina|     1|     GK|     GUZMAN Nahuel|    GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|  1|
|Argentina|    16|     DF|       ROJO Marcos|      ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20|  1|
|Argentina|    12|     GK|     ARMANI Franco|    ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16|  1|
|Argentina|    23|     GK|CABALLERO Wilfredo| CABALLERO|    Chelsea FC (ENG)|   186|  80| 28| 09|1981|     1981-09-28|  1|
|Argentina|     

Window analytic Functions ( Aqui, agora a gente entra em funções analíticas)


Vamos começar com o Window Function Lag/Degrau -lead()


In [26]:
df = df.drop('Nome_FIFA')

In [28]:
DEGRAU = Window.partitionBy('Selecao').orderBy(desc('Altura'))

df.withColumn('degrau', lag('Peso').over(DEGRAU)).show(50)

+---------+------+-------+----------+--------------------+------+----+---+---+----+---------------+------+
|  Selecao|Numero|Posicao|    Camisa|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|degrau|
+---------+------+-------+----------+--------------------+------+----+---+---+----+---------------+------+
|Argentina|     6|     DF|     FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|  NULL|
|Argentina|     1|     GK|    GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|    85|
|Argentina|    16|     DF|      ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20|    90|
|Argentina|    12|     GK|    ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16|    82|
|Argentina|    23|     GK| CABALLERO|    Chelsea FC (ENG)|   186|  80| 28| 09|1981|     1981-09-28|    85|
|Argentina|     9|     FW|   HIGUAÍN|   Juventus FC (ITA)|   184|  75| 10| 12|1987|     1987-12-10|    80|
|Argentina|     4|     DF|   ANSALDI|

In [29]:
DEGRAU = Window.partitionBy('Selecao').orderBy(desc('Altura'))

df.withColumn('degrau', lag('Altura').over(DEGRAU)).show(50)

+---------+------+-------+----------+--------------------+------+----+---+---+----+---------------+------+
|  Selecao|Numero|Posicao|    Camisa|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|degrau|
+---------+------+-------+----------+--------------------+------+----+---+---+----+---------------+------+
|Argentina|     6|     DF|     FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|  NULL|
|Argentina|     1|     GK|    GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|   199|
|Argentina|    16|     DF|      ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20|   192|
|Argentina|    12|     GK|    ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16|   189|
|Argentina|    23|     GK| CABALLERO|    Chelsea FC (ENG)|   186|  80| 28| 09|1981|     1981-09-28|   189|
|Argentina|     9|     FW|   HIGUAÍN|   Juventus FC (ITA)|   184|  75| 10| 12|1987|     1987-12-10|   186|
|Argentina|     4|     DF|   ANSALDI|

O lead é exatamente o contrário do LAG


In [33]:
degrau = Window.partitionBy('Selecao').orderBy(desc('Altura'))
df.withColumn('degrau', lead('Altura').over(degrau)).show(50)

+---------+------+-------+----------+--------------------+------+----+---+---+----+---------------+------+
|  Selecao|Numero|Posicao|    Camisa|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|degrau|
+---------+------+-------+----------+--------------------+------+----+---+---+----+---------------+------+
|Argentina|     6|     DF|     FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|   192|
|Argentina|     1|     GK|    GUZMÁN|   Tigres UANL (MEX)|   192|  90| 10| 02|1986|     1986-02-10|   189|
|Argentina|    16|     DF|      ROJO|Manchester United...|   189|  82| 20| 03|1990|     1990-03-20|   189|
|Argentina|    12|     GK|    ARMANI|CA River Plate (ARG)|   189|  85| 16| 10|1986|     1986-10-16|   186|
|Argentina|    23|     GK| CABALLERO|    Chelsea FC (ENG)|   186|  80| 28| 09|1981|     1981-09-28|   184|
|Argentina|     9|     FW|   HIGUAÍN|   Juventus FC (ITA)|   184|  75| 10| 12|1987|     1987-12-10|   181|
|Argentina|     4|     DF|   ANSALDI|

AGREGAÇÕES ! O GroupBy é muito parecido com o do Pandas. É bem simples. Você usa o " df.groupBy('*Aqui você insere o nome da coluna que você quer mexer*').agg({'*Nome do iten que vai ser agregado* : *Nome da definição que você quer'})

In [36]:
df.groupBy('Selecao').agg({'Altura':'avg'}).show()

+--------------+------------------+
|       Selecao|       avg(Altura)|
+--------------+------------------+
|        Russia| 184.3913043478261|
|       Senegal|183.65217391304347|
|        Sweden| 185.7391304347826|
|       IR Iran|184.47826086956522|
|       Germany| 185.7826086956522|
|        France|183.30434782608697|
|     Argentina|178.43478260869566|
|       Belgium|185.34782608695653|
|          Peru| 177.6086956521739|
|       Croatia| 185.2608695652174|
|       Nigeria|184.52173913043478|
|Korea Republic| 181.8695652173913|
|         Spain|179.91304347826087|
|       Denmark| 186.6086956521739|
|       Morocco|182.69565217391303|
|        Panama|182.17391304347825|
|       Iceland|185.52173913043478|
|       Uruguay|181.04347826086956|
|        Mexico| 179.7826086956522|
|       Tunisia|183.08695652173913|
+--------------+------------------+
only showing top 20 rows



Como ele me jogou do menor para o maior, eu vou por aqui do maior para o menor

In [40]:
df.groupBy('Selecao').agg({'Altura':'avg'}).orderBy('avg(Altura)', ascending = False).show(50)


+--------------+------------------+
|       Selecao|       avg(Altura)|
+--------------+------------------+
|        Serbia|186.69565217391303|
|       Denmark| 186.6086956521739|
|       Germany| 185.7826086956522|
|        Sweden| 185.7391304347826|
|       Iceland|185.52173913043478|
|       Belgium|185.34782608695653|
|       Croatia| 185.2608695652174|
|       Nigeria|184.52173913043478|
|       IR Iran|184.47826086956522|
|        Russia| 184.3913043478261|
|       Senegal|183.65217391304347|
|        France|183.30434782608697|
|        Poland|183.17391304347825|
|       Tunisia|183.08695652173913|
|   Switzerland|182.91304347826087|
|       England| 182.7391304347826|
|       Morocco|182.69565217391303|
|        Panama|182.17391304347825|
|Korea Republic| 181.8695652173913|
|       Uruguay|181.04347826086956|
|         Egypt|             181.0|
|     Australia| 180.8695652173913|
|        Brazil| 180.7826086956522|
|      Colombia| 180.7826086956522|
|    Costa Rica|180.69565217

A Segunda forma é um pouco mais fácil:

In [55]:
df.groupBy('Selecao').agg({'Altura': 'max'}).orderBy('max(Altura)', ascending=False).show(50)

+--------------+-----------+
|       Selecao|max(Altura)|
+--------------+-----------+
|       Croatia|        201|
|       Denmark|        200|
|     Argentina|        199|
|       Belgium|        199|
|        Sweden|        198|
|       Iceland|        198|
|        France|        197|
|       Nigeria|        197|
|Korea Republic|        197|
|        Panama|        197|
|        Russia|        196|
|       Senegal|        196|
|       Uruguay|        196|
|       England|        196|
|    Costa Rica|        196|
|       Germany|        195|
|        Brazil|        195|
|        Poland|        195|
|        Serbia|        195|
|       IR Iran|        194|
|         Spain|        194|
|         Egypt|        194|
|      Colombia|        194|
|     Australia|        193|
|       Tunisia|        192|
|  Saudi Arabia|        192|
|   Switzerland|        192|
|      Portugal|        191|
|       Morocco|        190|
|        Mexico|        190|
|          Peru|        189|
|         Japa

"Where " - o Where aqui é aquela fusão entre o pandas e o SQL também...

In [50]:
df.where('Selecao = "Brazil"').where('Posicao = "FW"').show()
# O where e o FIlter são muito parecidos
#where((condicao 1 ) & (condicao 2))

+-------+------+-------+---------+--------------------+------+----+---+---+----+---------------+
|Selecao|Numero|Posicao|   Camisa|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|
+-------+------+-------+---------+--------------------+------+----+---+---+----+---------------+
| Brazil|    21|     FW|   TAISON|FC Shakhtar Donet...|   172|  64| 13| 01|1988|     1988-01-13|
| Brazil|    10|     FW|NEYMAR JR|Paris Saint-Germa...|   175|  68| 05| 02|1992|     1992-02-05|
| Brazil|     7|     FW| D. COSTA|   Juventus FC (ITA)|   182|  70| 14| 09|1990|     1990-09-14|
| Brazil|     9|     FW| G. JESUS|Manchester City F...|   175|  73| 03| 04|1997|     1997-04-03|
| Brazil|    20|     FW|  FIRMINO|  Liverpool FC (ENG)|   181|  76| 02| 10|1991|     1991-10-02|
+-------+------+-------+---------+--------------------+------+----+---+---+----+---------------+



Se a gente quiser saber todas as informações do jogador mais alto, por exemplo, temos aqui a opção de pegar o

In [60]:
top1 = Window.partitionBy('Selecao').orderBy(desc('Altura'))

df.withColumn('Top',rank().over(top1)).where('Top = 1').show(50)

+--------------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+---+
|       Selecao|Numero|Posicao|       Camisa|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|Top|
+--------------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+---+
|     Argentina|     6|     DF|        FAZIO|       AS Roma (ITA)|   199|  85| 17| 03|1987|     1987-03-17|  1|
|     Australia|    12|     GK|        JONES|Feyenoord Rotterd...|   193|  87| 19| 03|1982|     1982-03-19|  1|
|       Belgium|     1|     GK|     COURTOIS|    Chelsea FC (ENG)|   199|  91| 11| 05|1992|     1992-05-11|  1|
|        Brazil|    16|     GK|       CASSIO|SC Corinthians (BRA)|   195|  92| 06| 06|1987|     1987-06-06|  1|
|      Colombia|    13|     DF|      Y. MINA|  FC Barcelona (ESP)|   194|  95| 23| 09|1994|     1994-09-23|  1|
|    Costa Rica|    19|     DF|    K. WASTON|Vancouver Whiteca...|   196|  87| 01| 01|1988|     1988-01-

Describe - Função para descrever todas as informações da tabela


In [61]:
df.describe().show()

+-------+---------+-----------------+-------+---------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|  Selecao|           Numero|Posicao|   Camisa|                Time|           Altura|             Peso|               Dia|               Mes|               Ano|
+-------+---------+-----------------+-------+---------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|  count|      736|              736|    736|      736|                 736|              736|              736|               736|               736|               736|
|   mean|     NULL|             12.0|   NULL|     NULL|                NULL|182.4076086956522|77.18885869565217|15.793478260869565|5.8790760869565215| 1990.110054347826|
| stddev|     NULL|6.637760461599851|   NULL|     NULL|                NULL|6.930924233929302|7.233778346883639| 8.761123828732469|3.3782493094684387|

In [64]:
df.where('Selecao = "Brazil"').describe().show(5)

+-------+-------+-----------------+-------+---------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|Selecao|           Numero|Posicao|   Camisa|                Time|           Altura|             Peso|              Dia|              Mes|              Ano|
+-------+-------+-----------------+-------+---------+--------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|     23|               23|     23|       23|                  23|               23|               23|               23|               23|               23|
|   mean|   NULL|             12.0|   NULL|     NULL|                NULL|180.7826086956522|76.56521739130434|11.26086956521739|6.130434782608695|1989.391304347826|
| stddev|   NULL|6.782329983125267|   NULL|     NULL|                NULL|7.354383490255254|8.239737898283606|6.876953549252538|2.784769418006175|3.499858833968506|
|    min| 

Função de agregação usando o Window Function

In [82]:
parametro = Window.partitionBy('Selecao').orderBy(desc('Altura'))
parametro2 = Window.partitionBy('Selecao')

dfmedias = df.withColumn('linhanova', row_number().over(parametro))\
.withColumn('media', avg('Altura').over(parametro2))\
.withColumn('maximo', max('Altura').over(parametro2))\
.withColumn('minimo', min('Altura').over(parametro2))\
.filter(col('linhanova') == "1")\
.select('Selecao','media','maximo','minimo')\
.orderBy('media', ascending = False)

In [83]:
dfmedias.show(50)

+--------------+------------------+------+------+
|       Selecao|             media|maximo|minimo|
+--------------+------------------+------+------+
|        Serbia|186.69565217391303|   195|   169|
|       Denmark| 186.6086956521739|   200|   171|
|       Germany| 185.7826086956522|   195|   176|
|        Sweden| 185.7391304347826|   198|   177|
|       Iceland|185.52173913043478|   198|   170|
|       Belgium|185.34782608695653|   199|   169|
|       Croatia| 185.2608695652174|   201|   172|
|       Nigeria|184.52173913043478|   197|   172|
|       IR Iran|184.47826086956522|   194|   177|
|        Russia| 184.3913043478261|   196|   173|
|       Senegal|183.65217391304347|   196|   173|
|        France|183.30434782608697|   197|   168|
|        Poland|183.17391304347825|   195|   172|
|       Tunisia|183.08695652173913|   192|   170|
|   Switzerland|182.91304347826087|   192|   165|
|       England| 182.7391304347826|   196|   170|
|       Morocco|182.69565217391303|   190|   167|
